In [1]:
!nvidia-smi

Mon Nov 24 18:55:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 573.24                 Driver Version: 573.24         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5080 ...  WDDM  |   00000000:02:00.0 Off |                  N/A |
| N/A   40C    P8              4W /   66W |     129MiB /  16303MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

CUDA available: True
GPU: NVIDIA GeForce RTX 5080 Laptop GPU


c:\Users\adewu\anaconda3\envs\aics_gpu\lib\site-packages\torch\cuda\__init__.py:235: UserWarning: 
NVIDIA GeForce RTX 5080 Laptop GPU with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_61 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5080 Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


In [3]:
!pip install -q torch torchvision torchaudio
!pip install -q rasterio scikit-learn
!pip install -q numpy matplotlib

In [4]:
!pip install pandas seaborn matplotlib numpy
!pip install opencv-python

In [5]:
import sys
print(sys.executable)

c:\Users\adewu\anaconda3\envs\aics_gpu\python.exe


In [6]:
import sys
!{sys.executable} -m pip install --force-reinstall --no-cache-dir opencv-python-headless

   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
   -- ------------------------------------- 2.4/38.9 MB 13.4 MB/s eta 0:00:03
   ----- ---------------------------------- 5.8/38.9 MB 14.1 MB/s eta 0:00:03
   --------- ------------------------------ 8.9/38.9 MB 14.2 MB/s eta 0:00:03
   ------------ --------------------------- 12.1/38.9 MB 14.2 MB/s eta 0:00:02
   --------------- ------------------------ 15.2/38.9 MB 14.3 MB/s eta 0:00:02
   ------------------ --------------------- 17.8/38.9 MB 14.1 MB/s eta 0:00:02
   --------------------- ------------------ 21.0/38.9 MB 14.3 MB/s eta 0:00:02
   ------------------------- -------------- 24.4/38.9 MB 14.4 MB/s eta 0:00:02
   ---------------------------- ----------- 27.5/38.9 MB 14.5 MB/s eta 0:00:01
   ------------------------------- -------- 30.9/38.9 MB 14.5 MB/s eta 0:00:01
   ---------------------------------- ----- 33.8/38.9 MB 14.5 MB/s eta 0:00:01
   ------------------------------------- -- 37.0/38.9 MB 14.6 MB

In [7]:
import sys
!{sys.executable} -m pip install tqdm

In [8]:
import sys
!{sys.executable} -m pip install gdown

In [11]:
!conda activate your_env
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install rasterio pillow matplotlib seaborn pandas scikit-learn tqdm opencv-python

Failed to run 'conda activate your_env'.



EnvironmentNameNotFound: Could not find conda environment: your_env
You can list all discoverable environments with `conda info --envs`.




Looking in indexes: https://download.pytorch.org/whl/cu118


In [12]:
"""
DRONE-BASED MULTISPECTRAL DISEASE DETECTION SYSTEM
==================================================
Adapted for VS Code / Local Machine with GPU

Key changes from Colab version:
- Uses local directory structure instead of /content/
- Removed gdown (use manual downloads)
- Added better path handling for Windows/Linux
- Optimized for local GPU usage
"""

import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torchvision.transforms.functional as TF

# Image Processing
import cv2
from PIL import Image
import rasterio
from rasterio.plot import show

# Machine Learning
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                             precision_recall_fscore_support, roc_auc_score,
                             matthews_corrcoef, cohen_kappa_score)
from sklearn.model_selection import train_test_split

# Utilities
from tqdm.auto import tqdm
import json
from datetime import datetime

# ============================================================================
# CONFIGURATION FOR LOCAL MACHINE
# ============================================================================

class Config:
    """Configuration adapted for local machine"""
    
    # Use current working directory or specify your own base path
    BASE_DIR = Path.cwd()  # Current working directory
    # Alternative: BASE_DIR = Path('D:/DroneAnalysis')  # Specify your own path
    
    # Local directory structure
    DATA_DIR = BASE_DIR / 'data'
    DRONE_DATA_DIR = DATA_DIR / 'drone_data'
    OUTPUT_DIR = BASE_DIR / 'output'
    MODEL_DIR = BASE_DIR / 'models'
    
    # Training parameters (optimized for local GPU)
    BATCH_SIZE = 8  # Reduced from 16 - adjust based on your VRAM
    EPOCHS = 15
    LEARNING_RATE = 0.001
    IMG_SIZE = 224
    NUM_WORKERS = 4  # Adjust based on your CPU cores
    
    # Model architecture
    MODEL_NAME = 'efficientnet_b0'
    NUM_CLASSES = 9
    
    # Vegetation indices thresholds
    NDVI_HEALTHY_MIN = 0.5
    NDVI_STRESS_MAX = 0.3
    STRESS_THRESHOLD = 0.4
    
    # Classes
    CLASSES = [
        'background', 'double_plant', 'drydown', 'endrow',
        'nutrient_deficiency', 'planter_skip', 'water',
        'waterway', 'weed_cluster'
    ]
    
    def __init__(self):
        # Create all necessary directories
        for dir_path in [self.DATA_DIR, self.DRONE_DATA_DIR, 
                         self.OUTPUT_DIR, self.MODEL_DIR]:
            dir_path.mkdir(parents=True, exist_ok=True)

config = Config()

# Check CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("="*70)
print("🚀 SYSTEM INFORMATION")
print("="*70)
print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("⚠️  WARNING: CUDA not available! Running on CPU")
    
print(f"\n📁 Working Directory: {config.BASE_DIR}")
print(f"📁 Output Directory: {config.OUTPUT_DIR}")
print(f"📁 Data Directory: {config.DRONE_DATA_DIR}")
print("="*70)

# ============================================================================
# MULTISPECTRAL IMAGE PROCESSOR
# ============================================================================

class MultispectralProcessor:
    """Process DJI Phantom 4 Multispectral TIF images"""
    
    def __init__(self):
        self.band_names = ['Blue', 'Green', 'Red', 'Red-edge', 'NIR']
    
    def load_tif(self, tif_path):
        """Load multispectral TIF and extract bands"""
        with rasterio.open(tif_path) as src:
            bands = src.read()
            profile = src.profile
            
            if bands.shape[0] != 5:
                raise ValueError(f"Expected 5 bands, got {bands.shape[0]}")
            
            # Normalize to 0-1
            bands = bands.astype(np.float32)
            for i in range(bands.shape[0]):
                band_min = bands[i].min()
                band_max = bands[i].max()
                if band_max > band_min:
                    bands[i] = (bands[i] - band_min) / (band_max - band_min)
            
            return bands, profile
    
    def calculate_ndvi(self, bands):
        """Calculate NDVI: (NIR - Red) / (NIR + Red)"""
        nir = bands[4]
        red = bands[2]
        ndvi = np.where((nir + red) == 0, 0, (nir - red) / (nir + red))
        return ndvi
    
    def calculate_gndvi(self, bands):
        """Calculate GNDVI: (NIR - Green) / (NIR + Green)"""
        nir = bands[4]
        green = bands[1]
        gndvi = np.where((nir + green) == 0, 0, (nir - green) / (nir + green))
        return gndvi
    
    def calculate_ndre(self, bands):
        """Calculate NDRE: (NIR - Red-edge) / (NIR + Red-edge)"""
        nir = bands[4]
        red_edge = bands[3]
        ndre = np.where((nir + red_edge) == 0, 0, (nir - red_edge) / (nir + red_edge))
        return ndre
    
    def calculate_stress_score(self, ndvi, ndre, gndvi):
        """Calculate vegetation stress score (0-1, higher = more stress)"""
        stress = (1 - ndvi) * 0.5 + (1 - ndre) * 0.3 + (1 - gndvi) * 0.2
        stress = np.clip(stress, 0, 1)
        return stress
    
    def detect_anomalies(self, stress_map, contamination=0.1):
        """Detect anomalous regions using Isolation Forest"""
        h, w = stress_map.shape
        X = stress_map.reshape(-1, 1)
        
        clf = IsolationForest(contamination=contamination, random_state=42)
        predictions = clf.fit_predict(X)
        
        anomaly_map = predictions.reshape(h, w)
        anomaly_map = (anomaly_map == -1).astype(float)
        
        return anomaly_map
    
    def cluster_vegetation(self, bands, n_clusters=5):
        """Segment vegetation into clusters using K-Means"""
        h, w = bands.shape[1], bands.shape[2]
        
        ndvi = self.calculate_ndvi(bands)
        ndre = self.calculate_ndre(bands)
        gndvi = self.calculate_gndvi(bands)
        
        features = np.stack([ndvi, ndre, gndvi], axis=-1)
        X = features.reshape(-1, 3)
        
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X)
        
        cluster_map = labels.reshape(h, w)
        return cluster_map, kmeans.cluster_centers_

processor = MultispectralProcessor()

# ============================================================================
# VISUALIZATION FUNCTIONS
# ============================================================================

def create_heatmap(data, title, cmap='RdYlGn', vmin=-1, vmax=1, figsize=(10, 8)):
    """Create a heatmap visualization"""
    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(data, cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.axis('off')
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout()
    return fig

def visualize_all_outputs(rgb_img, bands, ndvi, gndvi, ndre, stress, anomalies, clusters, output_path):
    """Create comprehensive visualization grid"""
    
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # RGB Reference
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.imshow(rgb_img)
    ax1.set_title('RGB Reference', fontsize=12, fontweight='bold')
    ax1.axis('off')
    
    # NDVI
    ax2 = fig.add_subplot(gs[0, 1])
    im2 = ax2.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
    ax2.set_title('NDVI', fontsize=12, fontweight='bold')
    ax2.axis('off')
    plt.colorbar(im2, ax=ax2, fraction=0.046)
    
    # GNDVI
    ax3 = fig.add_subplot(gs[0, 2])
    im3 = ax3.imshow(gndvi, cmap='RdYlGn', vmin=-1, vmax=1)
    ax3.set_title('GNDVI', fontsize=12, fontweight='bold')
    ax3.axis('off')
    plt.colorbar(im3, ax=ax3, fraction=0.046)
    
    # NDRE
    ax4 = fig.add_subplot(gs[1, 0])
    im4 = ax4.imshow(ndre, cmap='RdYlGn', vmin=-1, vmax=1)
    ax4.set_title('NDRE', fontsize=12, fontweight='bold')
    ax4.axis('off')
    plt.colorbar(im4, ax=ax4, fraction=0.046)
    
    # Stress Map
    ax5 = fig.add_subplot(gs[1, 1])
    im5 = ax5.imshow(stress, cmap='hot', vmin=0, vmax=1)
    ax5.set_title('Vegetation Stress', fontsize=12, fontweight='bold')
    ax5.axis('off')
    plt.colorbar(im5, ax=ax5, fraction=0.046)
    
    # Anomalies
    ax6 = fig.add_subplot(gs[1, 2])
    im6 = ax6.imshow(anomalies, cmap='Reds', vmin=0, vmax=1)
    ax6.set_title('Anomalies', fontsize=12, fontweight='bold')
    ax6.axis('off')
    plt.colorbar(im6, ax=ax6, fraction=0.046)
    
    # Clusters
    ax7 = fig.add_subplot(gs[2, 0])
    im7 = ax7.imshow(clusters, cmap='tab10')
    ax7.set_title('Vegetation Clusters', fontsize=12, fontweight='bold')
    ax7.axis('off')
    plt.colorbar(im7, ax=ax7, fraction=0.046)
    
    # Statistics
    ax8 = fig.add_subplot(gs[2, 1:])
    ax8.axis('off')
    
    stats_text = f"""
VEGETATION HEALTH STATISTICS

NDVI Stats:
  Mean:    {ndvi.mean():.3f}
  Std Dev: {ndvi.std():.3f}
  Range:   [{ndvi.min():.3f}, {ndvi.max():.3f}]

Stress Analysis:
  Avg Stress:    {stress.mean():.3f}
  High Stress %: {(stress > 0.6).sum() / stress.size * 100:.2f}%
  Anomaly %:     {anomalies.sum() / anomalies.size * 100:.2f}%

Health Assessment:
  Healthy (>0.5):    {(ndvi > 0.5).sum() / ndvi.size * 100:.1f}%
  Moderate (0.3-0.5): {((ndvi >= 0.3) & (ndvi <= 0.5)).sum() / ndvi.size * 100:.1f}%
  Stressed (<0.3):   {(ndvi < 0.3).sum() / ndvi.size * 100:.1f}%
    """
    
    ax8.text(0.1, 0.5, stats_text, fontsize=11, family='monospace',
             verticalalignment='center', 
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()

# ============================================================================
# SEQUENTIAL BAND PROCESSOR (FOR DJI PHANTOM 4 MULTISPECTRAL)
# ============================================================================

def process_sequential_bands(zip_path):
    """
    Process DJI Phantom 4 Multispectral data where bands are sequential
    """
    
    print("\n" + "="*70)
    print("🚁 SEQUENTIAL BAND PROCESSOR")
    print("="*70)
    
    zip_path = Path(zip_path)
    
    if not zip_path.exists():
        print(f"❌ ERROR: File not found: {zip_path}")
        print(f"   Make sure the file exists at: {zip_path.absolute()}")
        return None
    
    # Extract ZIP
    print(f"\n📦 Extracting {zip_path.name}...")
    extract_dir = config.DRONE_DATA_DIR / zip_path.stem
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"✓ Extracted to: {extract_dir}")
    
    # Find all files
    all_tifs = sorted(list(extract_dir.rglob('*.TIF')) + list(extract_dir.rglob('*.tif')))
    all_jpgs = sorted(list(extract_dir.rglob('*.JPG')) + list(extract_dir.rglob('*.jpg')))
    
    print(f"\n📊 Found:")
    print(f"   • {len(all_tifs)} TIF files")
    print(f"   • {len(all_jpgs)} JPG files")
    print(f"   • Expected images: {len(all_tifs) // 5}")
    
    if len(all_tifs) == 0:
        print("❌ No TIF files found!")
        return None
    
    # Group files: JPG at X, then TIFs at X+1 to X+5
    print(f"\n🔧 Grouping files...")
    
    tif_dict = {int(''.join(filter(str.isdigit, f.stem))): f for f in all_tifs}
    jpg_dict = {int(''.join(filter(str.isdigit, f.stem))): f for f in all_jpgs}
    
    image_groups = []
    for jpg_num in sorted(jpg_dict.keys()):
        band_files = []
        for offset in range(1, 6):
            tif_num = jpg_num + offset
            if tif_num in tif_dict:
                band_files.append(tif_dict[tif_num])
        
        if len(band_files) == 5:
            image_groups.append({
                'bands': band_files,
                'jpg_file': jpg_dict[jpg_num],
                'image_num': jpg_num
            })
    
    print(f"✓ Created {len(image_groups)} complete image groups")
    
    # Combine bands
    print(f"\n📦 Combining bands into multispectral images...")
    combined_dir = extract_dir / 'combined_multispectral'
    combined_dir.mkdir(exist_ok=True)
    
    combined_files = []
    
    for group in tqdm(image_groups, desc="Combining bands"):
        try:
            band_arrays = []
            profile = None
            
            for band_file in group['bands']:
                with rasterio.open(band_file) as src:
                    band_data = src.read(1)
                    band_arrays.append(band_data)
                    if profile is None:
                        profile = src.profile.copy()
            
            combined = np.stack(band_arrays, axis=0)
            
            profile.update({'count': 5, 'dtype': combined.dtype})
            
            jpg_base = group['jpg_file'].stem
            output_path = combined_dir / f"{jpg_base}_MS.TIF"
            
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(combined)
            
            combined_files.append(output_path)
            
        except Exception as e:
            print(f"❌ Error combining image {group['image_num']}: {e}")
            continue
    
    print(f"✓ Successfully combined {len(combined_files)} images")
    
    # Process all combined images
    print(f"\n{'='*70}")
    print(f"🚁 Processing {len(combined_files)} multispectral images...")
    print(f"{'='*70}\n")
    
    results = []
    
    for idx, tif_path in enumerate(combined_files):
        try:
            print(f"{'─'*70}")
            print(f"[{idx+1}/{len(combined_files)}] {tif_path.stem}")
            print(f"{'─'*70}")
            
            img_output_dir = config.OUTPUT_DIR / tif_path.stem
            img_output_dir.mkdir(exist_ok=True)
            
            # Load bands
            bands, profile = processor.load_tif(tif_path)
            
            # Calculate indices
            ndvi = processor.calculate_ndvi(bands)
            gndvi = processor.calculate_gndvi(bands)
            ndre = processor.calculate_ndre(bands)
            stress = processor.calculate_stress_score(ndvi, ndre, gndvi)
            anomalies = processor.detect_anomalies(stress)
            clusters, centers = processor.cluster_vegetation(bands)
            
            print(f"✓ NDVI: {ndvi.mean():.3f} | Stress: {stress.mean():.3f} | Anomalies: {anomalies.sum()/anomalies.size*100:.1f}%")
            
            # Find matching JPG
            rgb_img = None
            try:
                jpg_path = [g['jpg_file'] for g in image_groups 
                           if combined_dir / f"{g['jpg_file'].stem}_MS.TIF" == tif_path][0]
                rgb_img = np.array(Image.open(jpg_path))
            except:
                rgb_img = np.stack([bands[4], bands[2], bands[1]], axis=-1)
                rgb_img = (rgb_img * 255).astype(np.uint8)
            
            # Generate visualizations
            viz_path = img_output_dir / 'complete_analysis.png'
            visualize_all_outputs(rgb_img, bands, ndvi, gndvi, ndre, stress,
                                anomalies, clusters, viz_path)
            
            create_heatmap(ndvi, 'NDVI', cmap='RdYlGn', vmin=-1, vmax=1)
            plt.savefig(img_output_dir / 'ndvi.png', dpi=150, bbox_inches='tight')
            plt.close()
            
            create_heatmap(stress, 'Stress', cmap='hot', vmin=0, vmax=1)
            plt.savefig(img_output_dir / 'stress.png', dpi=150, bbox_inches='tight')
            plt.close()
            
            # Generate report
            report = f"""
VEGETATION HEALTH ANALYSIS REPORT
{'='*70}
File: {tif_path.name}
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
{'='*70}

VEGETATION INDICES:
    NDVI:  Mean={ndvi.mean():.4f}, Std={ndvi.std():.4f}, Range=[{ndvi.min():.4f}, {ndvi.max():.4f}]
    GNDVI: Mean={gndvi.mean():.4f}
    NDRE:  Mean={ndre.mean():.4f}

STRESS ANALYSIS:
    Average Stress:    {stress.mean():.4f}
    High Stress:       {(stress > 0.6).sum() / stress.size * 100:.2f}%
    Moderate Stress:   {((stress >= 0.4) & (stress <= 0.6)).sum() / stress.size * 100:.2f}%
    Low Stress:        {(stress < 0.4).sum() / stress.size * 100:.2f}%

HEALTH CLASSIFICATION:
    Healthy:    {(ndvi > config.NDVI_HEALTHY_MIN).sum() / ndvi.size * 100:.2f}%
    Moderate:   {((ndvi >= config.NDVI_STRESS_MAX) & (ndvi <= config.NDVI_HEALTHY_MIN)).sum() / ndvi.size * 100:.2f}%
    Stressed:   {(ndvi < config.NDVI_STRESS_MAX).sum() / ndvi.size * 100:.2f}%

ANOMALIES: {anomalies.sum() / anomalies.size * 100:.2f}%
CLUSTERS:  {len(centers)}

ASSESSMENT:
"""
            if ndvi.mean() > config.NDVI_HEALTHY_MIN:
                report += "    ✓ Vegetation health: GOOD\n"
            elif ndvi.mean() > config.NDVI_STRESS_MAX:
                report += "    ⚠ Vegetation health: MODERATE\n"
            else:
                report += "    ❌ Vegetation health: STRESSED\n"
            
            report += f"\nOutput: {img_output_dir}\n{'='*70}\n"
            
            with open(img_output_dir / 'report.txt', 'w') as f:
                f.write(report)
            
            results.append({
                'filename': tif_path.name,
                'ndvi_mean': float(ndvi.mean()),
                'stress_mean': float(stress.mean()),
                'healthy_percent': float((ndvi > config.NDVI_HEALTHY_MIN).sum() / ndvi.size * 100),
                'output_dir': str(img_output_dir)
            })
            
        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    # Summary
    if results:
        df = pd.DataFrame(results)
        csv_path = config.OUTPUT_DIR / 'batch_results.csv'
        df.to_csv(csv_path, index=False)
        
        print(f"\n{'='*70}")
        print("✅ PROCESSING COMPLETE!")
        print(f"{'='*70}")
        print(f"📊 Processed: {len(results)} images")
        print(f"📁 Output: {config.OUTPUT_DIR}")
        print(f"📈 Avg NDVI: {df['ndvi_mean'].mean():.3f}")
        print(f"📈 Avg Stress: {df['stress_mean'].mean():.3f}")
        print(f"✓ Results saved to: {csv_path}")
        print("="*70)
    
    return results

# ============================================================================
# USAGE INSTRUCTIONS
# ============================================================================

print("""
╔══════════════════════════════════════════════════════════════════════╗
║              🚁 VS CODE / LOCAL MACHINE USAGE                        ║
╚══════════════════════════════════════════════════════════════════════╝

SETUP:
1. Place your ZIP file in a known location
2. Run the script from VS Code

USAGE:
# Update the path to your ZIP file location
results = process_sequential_bands('path/to/your/103FPLAN.zip')

# Example for Windows:
results = process_sequential_bands(r'C:/Users/YourName/Downloads/103FPLAN.zip')

# Example for relative path (if ZIP is in same folder as script):
results = process_sequential_bands('./103FPLAN.zip')

OUTPUTS:
All results will be saved in the 'output' folder next to your script:
  📂 output/
      ├── DJI_0XXX_MS/
      │   ├── complete_analysis.png
      │   ├── ndvi.png
      │   ├── stress.png
      │   └── report.txt
      └── batch_results.csv

GPU OPTIMIZATION TIPS:
- BATCH_SIZE is set to 8 (adjust in Config if needed)
- Monitor GPU usage with: nvidia-smi
- If out of memory, reduce BATCH_SIZE to 4 or 2

═══════════════════════════════════════════════════════════════════════
""")

🚀 SYSTEM INFORMATION
Device: cuda
PyTorch version: 2.5.1
GPU: NVIDIA GeForce RTX 5080 Laptop GPU
CUDA Memory: 17.09 GB
CUDA Version: 12.4

📁 Working Directory: c:\Users\adewu\Downloads
📁 Output Directory: c:\Users\adewu\Downloads\output
📁 Data Directory: c:\Users\adewu\Downloads\data\drone_data

╔══════════════════════════════════════════════════════════════════════╗
║              🚁 VS CODE / LOCAL MACHINE USAGE                        ║
╚══════════════════════════════════════════════════════════════════════╝

SETUP:
1. Place your ZIP file in a known location
2. Run the script from VS Code

USAGE:
# Update the path to your ZIP file location
results = process_sequential_bands('path/to/your/103FPLAN.zip')

# Example for Windows:
results = process_sequential_bands(r'C:/Users/YourName/Downloads/103FPLAN.zip')

# Example for relative path (if ZIP is in same folder as script):
results = process_sequential_bands('./103FPLAN.zip')

OUTPUTS:
All results will be saved in the 'output' folder next

In [14]:
# In VS Code Python file or Jupyter notebook:
results = process_sequential_bands(r'C:\Users\adewu\Documents\DCIM\103FPLAN.zip')


🚁 SEQUENTIAL BAND PROCESSOR

📦 Extracting 103FPLAN.zip...
✓ Extracted to: c:\Users\adewu\Downloads\data\drone_data\103FPLAN

📊 Found:
   • 980 TIF files
   • 196 JPG files
   • Expected images: 196

🔧 Grouping files...
✓ Created 98 complete image groups

📦 Combining bands into multispectral images...


Combining bands: 100%|██████████| 98/98 [00:14<00:00,  6.87it/s]


✓ Successfully combined 98 images

🚁 Processing 98 multispectral images...

──────────────────────────────────────────────────────────────────────
[1/98] DJI_0010_MS
──────────────────────────────────────────────────────────────────────
✓ NDVI: 0.216 | Stress: 0.814 | Anomalies: 10.0%
──────────────────────────────────────────────────────────────────────
[2/98] DJI_0020_MS
──────────────────────────────────────────────────────────────────────
✓ NDVI: 0.205 | Stress: 0.818 | Anomalies: 10.0%
──────────────────────────────────────────────────────────────────────
[3/98] DJI_0030_MS
──────────────────────────────────────────────────────────────────────
✓ NDVI: 0.218 | Stress: 0.813 | Anomalies: 10.0%
──────────────────────────────────────────────────────────────────────
[4/98] DJI_0040_MS
──────────────────────────────────────────────────────────────────────
✓ NDVI: 0.246 | Stress: 0.798 | Anomalies: 10.0%
──────────────────────────────────────────────────────────────────────
[5/98] DJI_00